In [3]:
import numpy as np
import pandas as pd

# Carga del archivo .npz
data = np.load("chb02_seizure_EEGwindow_1.npz")
eeg_data = data['EEG_win']  # Array de ventanas [num_ventanas, 21, 128]

# Carga de metadatos
metadata = pd.read_parquet("chb02_seizure_metadata_1.parquet")

# Exploración inicial
print(f"Shape de EEG data: {eeg_data.shape}")
print("Metadatos:")
print(metadata.head())


# Dividir ventanas según la clase
seizure_windows = eeg_data[metadata['class'] == 1]
non_seizure_windows = eeg_data[metadata['class'] == 0]

print(f"Ventanas con crisis: {len(seizure_windows)}")
print(f"Ventanas sin crisis: {len(non_seizure_windows)}")



import matplotlib.pyplot as plt

# Visualizar un canal específico de una ventana
window_idx = 0
channel_idx = 0  # Cambia para explorar diferentes canales

plt.plot(eeg_data[window_idx, channel_idx, :])
plt.title(f"Ventana {window_idx}, Canal {channel_idx}")
plt.xlabel("Muestras")
plt.ylabel("Amplitud")
plt.show()


BadZipFile: File is not a zip file

# **OTRA FORMA**

In [ ]:
import os
import numpy as np
import pandas as pd

def load_eeg_data(folder_path, metadata_path, patient_list=[], sample_size=-1, include_seizures=True, prints=False):
    """
    Carga los datos de EEG de archivos .npz y metadatos en formato parquet.

    Args:
        folder_path (str): Ruta de la carpeta con archivos .npz.
        metadata_path (str): Ruta del archivo parquet con los metadatos.
        patient_list (list, optional): IDs de los pacientes a incluir. Si está vacío, carga todos.
        sample_size (int, optional): Número máximo de ventanas por paciente. -1 para cargar todas.
        include_seizures (bool, optional): True para incluir ventanas con crisis, False para excluirlas.
        prints (bool, optional): Si True, imprime progreso.

    Returns:
        tuple:
            - eeg_data (list): Lista de arrays con señales EEG.
            - labels (list): Lista de etiquetas binarias (0: no crisis, 1: crisis).
            - patient_ids (list): IDs de pacientes correspondientes a cada ventana.
    """
    # Cargar metadatos
    metadata = pd.read_parquet(metadata_path)

    eeg_data = []
    labels = []
    patient_ids = []
    incorrect_shape = 0

    # Si no se proporciona una lista de pacientes, incluir todos los pacientes del metadata
    if not patient_list:
        patient_list = metadata['filename'].apply(lambda x: x.split("_")[0]).unique().tolist()

    # Iterar sobre cada paciente en la lista
    for i, patient_id in enumerate(patient_list):
        # Seleccionar las filas correspondientes al paciente
        patient_metadata = metadata[metadata['filename'].str.startswith(patient_id)]

        # Obtener los archivos .npz correspondientes al paciente
        npz_files = [f for f in os.listdir(folder_path) if f.startswith(patient_id) and f.endswith(".npz")]

        if prints:
            print(f"Cargando paciente {i+1}/{len(patient_list)}: {patient_id}")

        for npz_file in npz_files:
            file_path = os.path.join(folder_path, npz_file)

            # Cargar datos del archivo .npz
            data = np.load(file_path)
            windows = data['EEG_win']  # Array [num_ventanas, 21, 128]

            # Filtrar metadatos del archivo específico
            file_metadata = patient_metadata[patient_metadata['filename'] == npz_file.replace(".npz", "")]

            # Filtrar ventanas según presencia de crisis
            if include_seizures:
                selected_rows = file_metadata[file_metadata['class'] == 1]
            else:
                selected_rows = file_metadata[file_metadata['class'] == 0]

            selected_indices = selected_rows.index.tolist()

            # Aplicar muestreo si es necesario
            if sample_size > 0 and len(selected_indices) > sample_size:
                selected_indices = np.random.choice(selected_indices, sample_size, replace=False)

            # Añadir ventanas y etiquetas
            for idx in selected_indices:
                eeg_data.append(windows[idx])
                labels.append(file_metadata.loc[idx, 'class'])
                patient_ids.append(patient_id)

    print(f'Resize necesario en {incorrect_shape} ventanas.')
    return eeg_data, labels, patient_ids


In [ ]:
folder_path = "ruta_a_datos_npz"
metadata_path = "ruta_a_metadatos.parquet"

# Cargar datos EEG
eeg_data, labels, patient_ids = load_eeg_data(
    folder_path=folder_path,
    metadata_path=metadata_path,
    patient_list=[],  # Cargar todos los pacientes
    sample_size=200,  # Cargar un máximo de 200 ventanas por paciente
    include_seizures=True,  # Incluir solo ventanas con crisis
    prints=True
)

# Verificar datos cargados
print(f"Ventanas cargadas: {len(eeg_data)}")
print(f"Etiquetas (crisis): {sum(labels)} de {len(labels)}")
